In [70]:
# Dependencies
import requests
import json
import pandas as pd
from sqlalchemy import create_engine

### Scrape Host country data

In [47]:
url = 'https://en.wikipedia.org/wiki/List_of_Olympic_Games_host_cities'

In [48]:
# Read in tables off url
tables = pd.read_html(url)

In [49]:
# Save table I want as dataframe
df = tables[2]
df.head()

,City,City.1,Country,Year,Continent,Summer,Winter,Opening ceremony,Closing ceremony,Ref
0,NaN,Athens,Greece,1896,Europe,NaN,NaN,6 April 1896,15 April 1896,NaN
1,NaN,Paris,France,1900,Europe,NaN,NaN,14 May 1900,28 October 1900,NaN
2,NaN,St. Louis[a],United States,1904,North America,NaN,NaN,1 July 1904,23 November 1904,NaN
3,NaN,London[b],United Kingdom,1908,Europe,NaN,NaN,27 April 1908,31 October 1908,NaN
4,NaN,Stockholm,Sweden,1912,Europe,NaN,NaN,6 July 1912,22 July 1912,NaN


In [50]:
# Drop unused columns
df = df.drop(['City', 'Summer', 'Winter', 'Ref'], axis=1)
df.head()

,City.1,Country,Year,Continent,Opening ceremony,Closing ceremony
0,Athens,Greece,1896,Europe,6 April 1896,15 April 1896
1,Paris,France,1900,Europe,14 May 1900,28 October 1900
2,St. Louis[a],United States,1904,North America,1 July 1904,23 November 1904
3,London[b],United Kingdom,1908,Europe,27 April 1908,31 October 1908
4,Stockholm,Sweden,1912,Europe,6 July 1912,22 July 1912


In [51]:
# Rename columns
df = df.rename(columns={'City.1': 'city', 'Country': 'country', 'Year': 'year', 'Continent': 'continent', 'Opening ceremony': 'opening_ceremony', 'Closing ceremony': 'closing_ceremony'})

In [52]:
# Delete footnote references
df['city'] = df['city'].str.replace("\[a]", "")
df['city'] = df['city'].str.replace("\[b]", "")
df['city'] = df['city'].str.replace("\[c]", "")
df['city'] = df['city'].str.replace("\[d]", "")
df['city'] = df['city'].str.replace("\[e]", "")
df['city'] = df['city'].str.replace("\[f]", "")
df['city'] = df['city'].str.replace("\[g]", "")
df['city'] = df['city'].str.replace("\[h]", "")
df['city'] = df['city'].str.replace("\[i]", "")
df['continent'] = df['continent'].str.replace("\[h]", "")

In [53]:
# Make new column
df['location'] = df['city'].str.cat(df['country'], sep=", ")

In [58]:
# Drop rows 
index_names = [5, 15, 16, 17, 18, 55, 56, 57, 58, 59, 60, 61]
df.drop(index_names, axis=0, inplace = True)

In [60]:
# Fix row that was doubled up by adding new rows
df.drop(24, axis=0, inplace = True)
df = df.append({'city': 'Melbourne', 'country': 'Australia', 'year': 1956, 'continent': 'Oceania', 'opening_ceremony': '22 November 1956', 'closing_ceremony': '8 December 1956', 'location': 'Melbourne, Australia'}, ignore_index=True)
df = df.append({'city': 'Stockholm', 'country': 'Sweden', 'year': 1956, 'continent': 'Europe', 'opening_ceremony': '10 June 1956', 'closing_ceremony': '17 June 1956', 'location': 'Stockholm, Sweden'}, ignore_index=True)


In [61]:
df = df.sort_values(by=['year']).reset_index(drop=True)
df.head(5)

,city,country,year,continent,opening_ceremony,closing_ceremony,location
0,Athens,Greece,1896,Europe,6 April 1896,15 April 1896,"Athens, Greece"
1,Paris,France,1900,Europe,14 May 1900,28 October 1900,"Paris, France"
2,St. Louis,United States,1904,North America,1 July 1904,23 November 1904,"St. Louis, United States"
3,London,United Kingdom,1908,Europe,27 April 1908,31 October 1908,"London, United Kingdom"
4,Stockholm,Sweden,1912,Europe,6 July 1912,22 July 1912,"Stockholm, Sweden"


In [62]:
df["lat"]=""
df["lng"]=""
df.head()

,city,country,year,continent,opening_ceremony,closing_ceremony,location,lat,lng
0,Athens,Greece,1896,Europe,6 April 1896,15 April 1896,"Athens, Greece",,
1,Paris,France,1900,Europe,14 May 1900,28 October 1900,"Paris, France",,
2,St. Louis,United States,1904,North America,1 July 1904,23 November 1904,"St. Louis, United States",,
3,London,United Kingdom,1908,Europe,27 April 1908,31 October 1908,"London, United Kingdom",,
4,Stockholm,Sweden,1912,Europe,6 July 1912,22 July 1912,"Stockholm, Sweden",,


### API Call to get latlng coordinates

In [63]:
# Add Geocoordinates (latitude/longitude) of cities
for index, row in df.iterrows():
    cities = row['location']
    target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={cities}&key={key}"
    cities_data = requests.get(target_url)
    cities_data = cities_data.json()
    df.loc[index, "lat"] = cities_data["results"][0]["geometry"]["location"]["lat"]
    df.loc[index, "lng"] = cities_data["results"][0]["geometry"]["location"]["lng"]
df.head()

,city,country,year,continent,opening_ceremony,closing_ceremony,location,lat,lng
0,Athens,Greece,1896,Europe,6 April 1896,15 April 1896,"Athens, Greece",37.9838,23.7275
1,Paris,France,1900,Europe,14 May 1900,28 October 1900,"Paris, France",48.8566,2.35222
2,St. Louis,United States,1904,North America,1 July 1904,23 November 1904,"St. Louis, United States",38.627,-90.1994
3,London,United Kingdom,1908,Europe,27 April 1908,31 October 1908,"London, United Kingdom",51.5074,-0.127758
4,Stockholm,Sweden,1912,Europe,6 July 1912,22 July 1912,"Stockholm, Sweden",59.3293,18.0686


In [65]:
# Rename dataframe
hostsDF = df

In [66]:
# Convert object to string
hostsDF['lat'] = hostsDF['lat'].astype(str)

In [67]:
# Convert object to string
hostsDF['lng'] = hostsDF['lng'].astype(str)

In [68]:
# Make new column to hold lat,lng
hostsDF['latlng'] = hostsDF['lat'].str.cat(hostsDF['lng'], sep=", ")

In [69]:
hostsDF.head()

,city,country,year,continent,opening_ceremony,closing_ceremony,location,lat,lng,latlng
0,Athens,Greece,1896,Europe,6 April 1896,15 April 1896,"Athens, Greece",37.9838096,23.7275388,"37.9838096, 23.7275388"
1,Paris,France,1900,Europe,14 May 1900,28 October 1900,"Paris, France",48.856614,2.3522219,"48.856614, 2.3522219"
2,St. Louis,United States,1904,North America,1 July 1904,23 November 1904,"St. Louis, United States",38.6270025,-90.19940419999999,"38.6270025, -90.19940419999999"
3,London,United Kingdom,1908,Europe,27 April 1908,31 October 1908,"London, United Kingdom",51.5073509,-0.1277583,"51.5073509, -0.1277583"
4,Stockholm,Sweden,1912,Europe,6 July 1912,22 July 1912,"Stockholm, Sweden",59.32932349999999,18.0685808,"59.32932349999999, 18.0685808"


### Connect to database and insert data

In [54]:
# Open a connection to the database
engine = create_engine('sqlite:///Olympics.sqlite', echo=True)
sqlite_connection = engine.connect()

2021-07-20 19:56:08,079 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-07-20 19:56:08,080 INFO sqlalchemy.engine.base.Engine ()
2021-07-20 19:56:08,083 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-07-20 19:56:08,086 INFO sqlalchemy.engine.base.Engine ()


In [55]:
# Insert table into database
sqlite_table = "hostsData"
hostsDF.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-07-20 19:57:31,388 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("hostsData")
2021-07-20 19:57:31,390 INFO sqlalchemy.engine.base.Engine ()
2021-07-20 19:57:31,399 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("hostsData")
2021-07-20 19:57:31,400 INFO sqlalchemy.engine.base.Engine ()
2021-07-20 19:57:31,403 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "hostsData" (
	"index" BIGINT, 
	city TEXT, 
	country TEXT, 
	year BIGINT, 
	continent TEXT, 
	opening_ceremony TEXT, 
	closing_ceremony TEXT, 
	location TEXT, 
	lat TEXT, 
	lng TEXT, 
	latlng TEXT
)


2021-07-20 19:57:31,404 INFO sqlalchemy.engine.base.Engine ()
2021-07-20 19:57:31,406 INFO sqlalchemy.engine.base.Engine COMMIT
2021-07-20 19:57:31,413 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_hostsData_index" ON "hostsData" ("index")
2021-07-20 19:57:31,417 INFO sqlalchemy.engine.base.Engine ()
2021-07-20 19:57:31,431 INFO sqlalchemy.engine.base.Engine COMMIT
2021-07-20 19:57:31,439 INFO sqlalche

In [56]:
# Close connection to database
sqlite_connection.close()

### Scrape IOC country code data and insert into database

In [71]:
ioc_url = 'https://en.wikipedia.org/wiki/List_of_IOC_country_codes'

In [72]:
# Read in tables off url
tables = pd.read_html(ioc_url)

In [73]:
codes_df = tables[0]

In [74]:
codes_df.head()

,Code,National Olympic Committee,Other codes used,Link
0,.mw-parser-output .monospaced{font-family:mono...,Afghanistan,NaN,[2]
1,ALB,Albania,NaN,[3]
2,ALG,Algeria,AGR (1964)AGL (1968 S) from Spanish Argelia,[4]
3,AND,Andorra,NaN,[5]
4,ANG,Angola,ANO (As referenced in IAAF World Championships...,[6]


In [75]:
# Drop unused columns
codes_df = codes_df.drop(['Other codes used', 'Link'], axis=1)

In [76]:
# Replace cell with mistake
codes_df.at[0,'Code']='AFG'

In [77]:
# Rename columns
codes_df = codes_df.rename(columns={'Code': 'code','National Olympic Committee': 'country_name'})

In [78]:
codes_df

,code,country_name
0,AFG,Afghanistan
1,ALB,Albania
2,ALG,Algeria
3,AND,Andorra
4,ANG,Angola
...,...,...
201,VIE,Vietnam
202,VIN,Saint Vincent and the Grenadines
203,YEM,Yemen
204,ZAM,Zambia


In [89]:
# Open a connection to the database
engine = create_engine('sqlite:///../Olympics.sqlite', echo=True)
sqlite_connection = engine.connect()

2021-07-21 19:37:03,663 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-07-21 19:37:03,664 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 19:37:03,666 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-07-21 19:37:03,667 INFO sqlalchemy.engine.base.Engine ()


In [90]:
# Insert table into database
sqlite_table = "iocData"
codes_df.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2021-07-21 19:37:09,430 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("iocData")
2021-07-21 19:37:09,431 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 19:37:09,433 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("iocData")
2021-07-21 19:37:09,434 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 19:37:09,436 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "iocData" (
	"index" BIGINT, 
	code TEXT, 
	country_name TEXT
)


2021-07-21 19:37:09,437 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 19:37:09,439 INFO sqlalchemy.engine.base.Engine COMMIT
2021-07-21 19:37:09,440 INFO sqlalchemy.engine.base.Engine CREATE INDEX "ix_iocData_index" ON "iocData" ("index")
2021-07-21 19:37:09,442 INFO sqlalchemy.engine.base.Engine ()
2021-07-21 19:37:09,444 INFO sqlalchemy.engine.base.Engine COMMIT
2021-07-21 19:37:09,447 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2021-07-21 19:37:09,450 INFO sqlalchemy.engine.base.Engine INSERT INTO "iocData" ("index", code, country_

In [91]:
# Close connection to database
sqlite_connection.close()